# Einstein summation notation

Rules:

- two parts, inputs and outputs separated by `->`
- letters represent axis - hence `ijk` is a 3-tensor
- sum over repeated letters in inputs
- sum over letters not in output

The result of an `einsum` can  generally be understood as a element-wise multiplication of two tensors (with broadcasting) followed by a sum over zero or more axes. It turns out that this is a surprisingly flexible way of executing many common array operations.

It is easier to understand with examples.

In [ ]:
import numpy as np

## Set up a vector and a matrix

In [ ]:
x = np.arange(1, 4)
x

In [ ]:
M = np.arange(1, 10).reshape(3,3)
M

## Working with vectors

In [ ]:
np.einsum('i->i', x)

In [ ]:
np.einsum('i->', x)

In [ ]:
np.sum(x)

In [ ]:
np.einsum('i,j->ij', x, x)

In [ ]:
np.outer(x, x)

In [ ]:
x[:,None] * x[None, :]

In [ ]:
np.einsum('i,j->ji', x, x)

In [ ]:
np.outer(x, x).T

In [ ]:
(x[:,None] * x[None, :]).T

In [ ]:
np.einsum('i,i->', x, x)

In [ ]:
np.inner(x, x)

In [ ]:
(x * x).sum()

In [ ]:
np.einsum('i,i->i', x, x)

In [ ]:
x * x

## Working with matrices

In [ ]:
np.einsum('ii->i', M)

In [ ]:
np.diag(M)

In [ ]:
np.einsum('ij->ji', M)

In [ ]:
np.transpose(M)

In [ ]:
np.dot(M, M)

In [ ]:
np.einsum('ij,jk->ik', M, M)

In [ ]:
np.einsum('ij,ji->', M, M)

In [ ]:
(M*M.T).sum()

In [ ]:
np.einsum('ij,ij->', M, M)

In [ ]:
(M*M).sum()

In [ ]:
np.einsum('ij,jk->ijk', M, M)

In [ ]:
M[...,None] * M

In [ ]:
np.einsum('ij,kl->ijkl', M, M)

In [ ]:
M[...,None,None] * M

In [ ]:
np.einsum('ij,kl->jkl', M, M)

In [ ]:
(M[:,:,None,None] * M).sum(0)

In [ ]:
np.einsum('ij,jk->ijk', M, M)

In [ ]:
M[...,None] * M

In [ ]:
np.einsum('ij,jk->ik', M, M)

In [ ]:
M@M

In [ ]:
(M[...,None] * M).sum(1)

### Matrices and vectors

In [ ]:
np.einsum('ij,j->i', M, x)

In [ ]:
np.dot(M, x)

In [ ]:
(M * x[None,:]).sum(1)

In [ ]:
np.einsum('ij,i->ij', M, x)

In [ ]:
M*x[:, None]

In [ ]:
np.einsum('ij,i->i', M, x)

In [ ]:
(M*x[:, None]).sum(1)

In [ ]:
np.einsum('ij,i->j', M, x)

In [ ]:
(M*x[:, None]).sum(0)

## Example: Pairwise distances

In [ ]:
A = np.arange(12).reshape(3,4)
B = np.arange(16).reshape(4,4)

In [ ]:
A

In [ ]:
B

`scipy.pdist` can calculate pairwise distance for a set of row vectors, but not between 2 different sets

In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
squareform(pdist(A))

In plain Python

In [ ]:
res = np.zeros([A.shape[0], A.shape[0]])
for i in range(A.shape[0]):
    for j in range(A.shape[0]):
        res[i,j] = ((A[i] - A[j]) @ (A[i] - A[j]))**0.5
res

We can calculate the Euclidean distance between our row vectors by broadcasting. Since we want all pairwise results, this is basically a generalized outer product where the operator is subtraction rather than multiplication. We recall that the regular outer product for a matrix `M` is given using broadcasting by `M[:,None] * M[None,:]`

In [ ]:
M = np.arange(3)
M

In [ ]:
M[:,None] * M[None, :]

The second dimension expansion is actually unnecssary since broadcasting rules take care of it.

In [ ]:
M[:, None] * M

We basically do the same thing for A

In [ ]:
A

In [ ]:
A[:,None,:]

In [ ]:
A.shape

In [ ]:
A[:,None,:].shape

In [ ]:
A_ = A[:,None,:] - A
A_

With redundant dimension expansion, to make everything explicit. This makes it clear that the `k` components of each vector are left as they are, which makes sense since we treat each row vector as a unit.

In [ ]:
A[:, None, :] - A[None, :, :]

In [ ]:
A_ * A_

In [ ]:
(A_ * A_).sum(axis=2)**0.5

By now, we know that the pattern (X * X).sum(axis=k) can be written more efficiently as an einsum.

In [ ]:
np.einsum('ijk,ijk->ij', A_, A_)**0.5

Now we can do the same for two different sets of same length vectors

In [ ]:
C_ = A[:,None,:] - B
C_

In [ ]:
np.einsum('ijk,ijk->ij', C_, C_)**0.5

In [ ]:
res = np.zeros((A.shape[0], B.shape[0]))
for i in range(A.shape[0]):
    for j in range(B.shape[0]):
        res[i,j] = ((A[i] - B[j]) @ (A[i] - B[j]))**0.5
res